<a href="https://colab.research.google.com/github/MoqiSheng/MoqiSheng.github.io/blob/main/kmeans0220.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

加载数据

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/kmeans0219/All_image.zip -d /content/data

Archive:  /content/drive/MyDrive/kmeans0219/All_image.zip
  inflating: /content/data/0.png     
  inflating: /content/data/1.png     
  inflating: /content/data/10.png    
  inflating: /content/data/100.png   
  inflating: /content/data/1000.png  
  inflating: /content/data/1001.png  
  inflating: /content/data/1002.png  
  inflating: /content/data/1003.png  
  inflating: /content/data/1004.png  
  inflating: /content/data/1005.png  
  inflating: /content/data/1006.png  
  inflating: /content/data/1007.png  
  inflating: /content/data/1008.png  
  inflating: /content/data/1009.png  
  inflating: /content/data/101.png   
  inflating: /content/data/1010.png  
  inflating: /content/data/1011.png  
  inflating: /content/data/1012.png  
  inflating: /content/data/1013.png  
  inflating: /content/data/1014.png  
  inflating: /content/data/1015.png  
  inflating: /content/data/1016.png  
  inflating: /content/data/1017.png  
  inflating: /content/data/1018.png  
  inflating: /content/data/101

去重

In [7]:
import os
import torch
from transformers import ViTFeatureExtractor, ViTModel
from torchvision import transforms
from PIL import Image
import numpy as np
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# 加载预训练的 google/vit-base-patch16-224-in21k 模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# 设置图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 加载图像并提取特征向量
def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)  # 扩展batch维度，并传输到GPU（如果可用）
        with torch.no_grad():
            outputs = model(image)
        cls_token = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()  # 获取CLS token的特征向量
        if cls_token is not None:
            print(f"Feature extracted from {image_path}")
        return cls_token
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# 计算余弦相似度
def compute_similarity_matrix(features_list):
    return cosine_similarity(features_list)

# 数据去重
def remove_duplicates(image_dir, output_dir, similarity_threshold=0.63):
    image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(".jpg") or img.endswith(".png")]
    features_list = []
    failed_images = []

    # 提取图像特征
    for img_path in tqdm(image_paths, desc="Extracting features"):
        features = extract_features(img_path)
        if features is not None:
            features_list.append(features)
        else:
            failed_images.append(img_path)

    # 如果没有成功提取任何特征，报错并返回空列表
    if not features_list:
        print("No features were extracted. Please check the image files.")
        return []

    # 计算余弦相似度矩阵
    similarity_matrix = compute_similarity_matrix(features_list)

    # 根据相似度阈值进行去重
    to_remove = set()
    for i in range(len(image_paths)):
        if i in to_remove:
            continue
        for j in range(i + 1, len(image_paths)):
            if similarity_matrix[i][j] > similarity_threshold:
                to_remove.add(j)

    # 保留没有被标记为删除的图像
    unique_image_paths = [image_paths[i] for i in range(len(image_paths)) if i not in to_remove]

    # 创建输出目录（如果不存在）
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 将去重后的图像复制到新的目录
    for img_path in unique_image_paths:
        shutil.copy(img_path, output_dir)

    # 输出失败的图像
    if failed_images:
        print(f"The following images failed to extract features: {failed_images}")

    return unique_image_paths

# 设置数据文件夹路径和输出文件夹路径
image_dir = './data'
output_dir = './data_deduplication'
unique_images = remove_duplicates(image_dir, output_dir, similarity_threshold=0.63)

# 输出去重后的图像路径
print(f"去重后的图像数量: {len(unique_images)}")
for img_path in unique_images:
    print(img_path)



/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Extracting features:   0%|          | 7/1518 [00:00<00:23, 65.28it/s]

Feature extracted from ./data/779.png
Feature extracted from ./data/86.png
Feature extracted from ./data/555.png
Feature extracted from ./data/817.png
Feature extracted from ./data/608.png
Feature extracted from ./data/1414.png
Feature extracted from ./data/294.png
Feature extracted from ./data/703.png
Feature extracted from ./data/74.png
Feature extracted from ./data/1322.png
Feature extracted from ./data/951.png
Feature extracted from ./data/165.png
Feature extracted from ./data/498.png


Extracting features:   1%|▏         | 21/1518 [00:00<00:24, 61.22it/s]

Feature extracted from ./data/1493.png
Feature extracted from ./data/193.png
Feature extracted from ./data/707.png
Feature extracted from ./data/1178.png
Feature extracted from ./data/650.png
Feature extracted from ./data/824.png
Feature extracted from ./data/1313.png
Feature extracted from ./data/1276.png
Feature extracted from ./data/922.png
Feature extracted from ./data/481.png
Feature extracted from ./data/129.png
Feature extracted from ./data/1350.png
Feature extracted from ./data/593.png


Extracting features:   2%|▏         | 35/1518 [00:00<00:23, 62.45it/s]

...

Feature extracted from ./data/1101.png
Feature extracted from ./data/881.png
Feature extracted from ./data/447.png
Feature extracted from ./data/811.png
Feature extracted from ./data/507.png
Feature extracted from ./data/12.png
Feature extracted from ./data/331.png
Feature extracted from ./data/464.png
Feature extracted from ./data/1167.png
Feature extracted from ./data/724.png
Feature extracted from ./data/1208.png
Feature extracted from ./data/821.png
Feature extracted from ./data/191.png
Feature extracted from ./data/647.png


Extracting features:  99%|█████████▉| 1510/1518 [00:22<00:00, 68.22it/s]

Feature extracted from ./data/746.png
Feature extracted from ./data/998.png
Feature extracted from ./data/1025.png
Feature extracted from ./data/95.png
Feature extracted from ./data/209.png
Feature extracted from ./data/1139.png
Feature extracted from ./data/1483.png
Feature extracted from ./data/1509.png
Feature extracted from ./data/1186.png
Feature extracted from ./data/1185.png
Feature extracted from ./data/1106.png
Feature extracted from ./data/115.png
Feature extracted from ./data/1114.png
Feature extracted from ./data/685.png
Feature extracted from ./data/1352.png


Extracting features: 100%|██████████| 1518/1518 [00:22<00:00, 68.10it/s]


Feature extracted from ./data/534.png
Feature extracted from ./data/297.png
Feature extracted from ./data/1299.png
Feature extracted from ./data/189.png
Feature extracted from ./data/1067.png
去重后的图像数量: 750
./data/779.png
./data/86.png
./data/555.png
./data/817.png
./data/608.png
./data/1414.png
./data/294.png
./data/703.png
./data/74.png
./data/951.png
./data/165.png
./data/498.png
./data/1493.png
./data/193.png
./data/707.png
./data/1178.png
./data/650.png
./data/824.png
./data/1313.png
./data/922.png
./data/481.png
./data/593.png
./data/0.png
./data/1449.png
./data/420.png
./data/71.png
./data/960.png
./data/194.png
./data/932.png
./data/968.png
./data/1311.png
./data/953.png
./data/1197.png
./data/885.png
./data/899.png
./data/1191.png
./data/798.png
./data/310.png
./data/714.png
./data/173.png
./data/570.png
./data/920.png
./data/958.png
./data/866.png
./data/874.png
./data/1396.png
./data/575.png
./data/759.png
./data/1416.png
./data/853.png
./data/620.png
./data/559.png
./data/99

In [6]:
!rm -r /content/data_deduplication

In [8]:
import shutil

# 定义要压缩的文件夹路径和目标ZIP文件路径
folder_path = '/content/data_deduplication'
zip_path = '/content/drive/MyDrive/kmeans0219/data_deduplication.zip'

# 压缩文件夹为ZIP文件
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

'/content/drive/MyDrive/kmeans0219/data_deduplication.zip'

k-means聚类

In [9]:
!unzip /content/drive/MyDrive/kmeans0219/data_deduplication.zip -d /content/data_deduplication

Archive:  /content/drive/MyDrive/kmeans0219/data_deduplication.zip
replace /content/data_deduplication/779.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [10]:
import os
import torch
from transformers import ViTFeatureExtractor, ViTModel
from torchvision import transforms
from PIL import Image
import numpy as np
import shutil
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd

# 加载预训练的 google/vit-base-patch16-224-in21k 模型
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k").to(device)
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# 设置图像预处理
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 加载图像并提取特征向量
def extract_features(image_path):
    try:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0).to(device)  # 扩展batch维度，并传输到GPU（如果可用）
        with torch.no_grad():
            outputs = model(image)
        cls_token = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()  # 获取CLS token的特征向量
        if cls_token is not None:
            print(f"Feature extracted from {image_path}")
        return cls_token
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# 从图像目录加载图像并提取所有图像的特征
def extract_all_features(image_dir):
    image_paths = [os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith(".jpg") or img.endswith(".png")]
    features_list = []
    for img_path in tqdm(image_paths, desc="Extracting features"):
        features = extract_features(img_path)
        if features is not None:
            features_list.append(features)
    return np.array(features_list), image_paths

# 聚类选取最具特点的图像
def select_top_images(image_dir, num_images=152, output_dir='./data_kmeans'):
    features_list, image_paths = extract_all_features(image_dir)
    if len(features_list) == 0:
        print("No features were extracted. Please check the image files.")
        return []

    # 使用KMeans聚类
    kmeans = KMeans(n_clusters=num_images, random_state=42)
    kmeans.fit(features_list)

    # 获取每个聚类的标签（即每张图片所属的聚类）
    labels = kmeans.labels_

    # 将聚类结果保存到CSV文件中
    clustering_result = {"Image Path": image_paths, "Cluster Label": labels}
    df = pd.DataFrame(clustering_result)
    df.to_csv("clustering_result.csv", index=False)
    print("聚类结果已保存到 clustering_result.csv")

    # 创建输出目录
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 根据聚类标签选取最具特点的图像
    selected_image_paths = []
    for cluster_id in range(num_images):
        cluster_images = df[df["Cluster Label"] == cluster_id]["Image Path"].tolist()
        # 选取每个聚类中最接近聚类中心的图像，或其他策略
        selected_image_paths.extend(cluster_images[:1])  # 在每个聚类中选取1张图像（可以根据需要调整选取策略）

    # 将选中的图像复制到新的文件夹
    for img_path in selected_image_paths:
        shutil.copy(img_path, output_dir)

    # 输出选取的图像路径
    print(f"选取的最具特点的图像数量: {len(selected_image_paths)}")
    for img_path in selected_image_paths:
        print(img_path)

    return selected_image_paths

# 设置数据文件夹路径
image_dir = './data_deduplication'

# 聚类选取152张最具特点的图像，并将它们保存到 data_kmeans 文件夹
selected_images = select_top_images(image_dir, num_images=152, output_dir='./data_kmeans')



/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Extracting features:   2%|▏         | 14/750 [00:00<00:11, 66.42it/s]

Feature extracted from ./data_deduplication/779.png
Feature extracted from ./data_deduplication/86.png
Feature extracted from ./data_deduplication/555.png
Feature extracted from ./data_deduplication/817.png
Feature extracted from ./data_deduplication/608.png
Feature extracted from ./data_deduplication/1414.png
Feature extracted from ./data_deduplication/294.png
Feature extracted from ./data_deduplication/703.png
Feature extracted from ./data_deduplication/74.png
Feature extracted from ./data_deduplication/951.png
Feature extracted from ./data_deduplication/165.png
Feature extracted from ./data_deduplication/498.png
Feature extracted from ./data_deduplication/1493.png
Feature extracted from ./data_deduplication/193.png


Extracting features:   4%|▎         | 28/750 [00:00<00:10, 67.35it/s]

Feature extracted from ./data_deduplication/707.png
Feature extracted from ./data_deduplication/1178.png
Feature extracted from ./data_deduplication/650.png
Feature extracted from ./data_deduplication/824.png
Feature extracted from ./data_deduplication/1313.png
Feature extracted from ./data_deduplication/922.png
Feature extracted from ./data_deduplication/481.png
Feature extracted from ./data_deduplication/593.png
Feature extracted from ./data_deduplication/0.png
Feature extracted from ./data_deduplication/1449.png
Feature extracted from ./data_deduplication/420.png
Feature extracted from ./data_deduplication/71.png
Feature extracted from ./data_deduplication/960.png
Feature extracted from ./data_deduplication/194.png


Extracting features:   6%|▌         | 42/750 [00:00<00:10, 67.58it/s]

...

Feature extracted from ./data_deduplication/1246.png
Feature extracted from ./data_deduplication/1438.png
Feature extracted from ./data_deduplication/1080.png
Feature extracted from ./data_deduplication/752.png
Feature extracted from ./data_deduplication/862.png
Feature extracted from ./data_deduplication/1339.png
Feature extracted from ./data_deduplication/727.png
Feature extracted from ./data_deduplication/22.png
Feature extracted from ./data_deduplication/1266.png
Feature extracted from ./data_deduplication/637.png
Feature extracted from ./data_deduplication/1506.png
Feature extracted from ./data_deduplication/1389.png
Feature extracted from ./data_deduplication/1161.png
Feature extracted from ./data_deduplication/1280.png
Feature extracted from ./data_deduplication/87.png


Extracting features:  99%|█████████▊| 740/750 [00:10<00:00, 69.42it/s]

Feature extracted from ./data_deduplication/1503.png
Feature extracted from ./data_deduplication/257.png
Feature extracted from ./data_deduplication/252.png
Feature extracted from ./data_deduplication/76.png
Feature extracted from ./data_deduplication/560.png
Feature extracted from ./data_deduplication/1134.png
Feature extracted from ./data_deduplication/992.png
Feature extracted from ./data_deduplication/912.png
Feature extracted from ./data_deduplication/1153.png
Feature extracted from ./data_deduplication/1371.png
Feature extracted from ./data_deduplication/1046.png
Feature extracted from ./data_deduplication/26.png
Feature extracted from ./data_deduplication/1101.png
Feature extracted from ./data_deduplication/12.png


Extracting features: 100%|██████████| 750/750 [00:10<00:00, 70.16it/s]


Feature extracted from ./data_deduplication/1167.png
Feature extracted from ./data_deduplication/1208.png
Feature extracted from ./data_deduplication/821.png
Feature extracted from ./data_deduplication/647.png
Feature extracted from ./data_deduplication/746.png
Feature extracted from ./data_deduplication/1114.png
Feature extracted from ./data_deduplication/685.png
Feature extracted from ./data_deduplication/297.png
Feature extracted from ./data_deduplication/1067.png
聚类结果已保存到 clustering_result.csv
选取的最具特点的图像数量: 152
./data_deduplication/761.png
./data_deduplication/874.png
./data_deduplication/650.png
./data_deduplication/608.png
./data_deduplication/254.png
./data_deduplication/310.png
./data_deduplication/882.png
./data_deduplication/456.png
./data_deduplication/322.png
./data_deduplication/866.png
./data_deduplication/1211.png
./data_deduplication/1449.png
./data_deduplication/772.png
./data_deduplication/282.png
./data_deduplication/414.png
./data_deduplication/420.png
./data_dedupl

In [ ]:
import shutil

# 定义要压缩的文件夹路径和目标ZIP文件路径
folder_path = '/content/data_kmeans'
zip_path = '/content/drive/MyDrive/kmeans0219/data_kmeans.zip'

# 压缩文件夹为ZIP文件
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)

'/content/drive/MyDrive/kmeans0219/data_kmeans.zip'